# Soft constraints variation

In [ ]:
!apt-get install -y glpk-utils
!pip install --upgrade pyomo

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 18 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libglpk40 amd64 5.0-1 [361 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/universe amd64 glpk-ut

In [ ]:
!which glpsol

/usr/bin/glpsol


In [ ]:
from pyomo.environ import SolverFactory

solver = SolverFactory("glpk", executable="/usr/bin/glpsol")
print(solver.available())  # Should return True

True


In [ ]:
import pyomo.environ as pyo
print(pyo.SolverFactory("glpk").available())

True


In [ ]:
# Re-import necessary libraries after execution state reset
import pandas as pd
from pyomo.environ import (
    ConcreteModel, Var, Objective, Constraint,
    NonNegativeReals, Binary, SolverFactory, minimize, value
)
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import os
os.listdir()

['.config', 'sample_data']

In [ ]:
!find /content -name "jrc_data_uk.csv"


In [ ]:
from google.colab import files

# Open file upload dialog
uploaded = files.upload()



Saving repurposing_cost.csv to repurposing_cost.csv
Saving patchwork_demand.csv to patchwork_demand.csv
Saving enhancements_cost.csv to enhancements_cost.csv
Saving decommission_cost.csv to decommission_cost.csv
Saving clockwork_demand.csv to clockwork_demand.csv


In [ ]:
# -------------------------------
# 1. DATA IMPORT & PREPARATION
# -------------------------------

#indir = "../Data_for_experiments/"

# Read powerplant data
jrc_data_uk = pd.read_csv("jrc_data_uk.csv", low_memory=False)

def capacity_p_dict(type_g):
    jrc_type_g = jrc_data_uk[jrc_data_uk["type_g"] == type_g]
    type_g_plants = jrc_type_g.set_index("eic_p")["capacity_p"].to_dict()
    return type_g_plants

# Define plant capacities for various technologies
gas_plants = capacity_p_dict("Fossil Gas")
coal_plants = capacity_p_dict("Fossil Hard coal")
wind_on_plants = capacity_p_dict("Wind Onshore")
hydro_river_plants = capacity_p_dict("Hydro Run-of-river and poundage")
wind_off_plants = capacity_p_dict("Wind Offshore")
biomass_plants = capacity_p_dict("Biomass")
nuclear_plants = capacity_p_dict("Nuclear")
oil_plants = capacity_p_dict("Fossil Oil")
# hydro_storage_plants = capacity_p_dict("Hydro Pumped Storage")

# (Optional) Dictionary mapping each plant identifier to its type
plant_types = jrc_data_uk.set_index("eic_p")["type_g"].to_dict()

# Read demand scenarios
clockwork_demand = pd.read_csv("clockwork_demand.csv", low_memory=False)
patchwork_demand = pd.read_csv("patchwork_demand.csv", low_memory=False)

# Read cost data
decommission_cost = pd.read_csv("decommission_cost.csv", low_memory=False)
enhancements_cost = pd.read_csv("enhancements_cost.csv", low_memory=False)
repurposing_cost = pd.read_csv("repurposing_cost.csv", low_memory=False)

In [ ]:
# Decommissioning costs per MW
decommissioning_costs_per_mw = decommission_cost.set_index("technology")["max unit cost (millions GBP/MW)"].to_dict()

# increase reneable plant cost per MW
enhancements_cost_per_mw = enhancements_cost.set_index("technology")["max unit cost (millions GBP/MW)"].to_dict()

# percentage of enhanced capacity as a % of original in Mw
# e.g. increase by 50 % is 0.5*old_capacity in MW
enhancement_percentage = {'Solar Panel': 0.5,
 'Hydro Run-of-river and poundage': 0.5,
 'Nuclear': 0.5,
 'Wind Offshore': 0.5,
 'Wind Onshore': 0.5,
 'Hydro Pumped Storage': 0.5}

#################################################################
# check again the loops etc, seems to work fine now

# Ensure the column names are correct
expected_columns = ["from", "to", "max unit cost (millions GBP/MW)"]
for col in expected_columns:
    if col not in repurposing_cost.columns:
        raise ValueError(f"Missing expected column: {col}")

# Extract the dictionary
repurposing_costs_per_mw = {}
for _, row in repurposing_cost.iterrows():
    from_source = row["from"]
    to_target = row["to"]
    cost = row["max unit cost (millions GBP/MW)"]

    # Ensure 'from' is in the dictionary
    if from_source not in repurposing_costs_per_mw:
        repurposing_costs_per_mw[from_source] = {}

    # Assign value if not NaN
    if not pd.isna(cost):
        repurposing_costs_per_mw[from_source][to_target] = cost


print(repurposing_costs_per_mw)

{'Fossil Hard coal': {'Biomass': 1.6, 'Nuclear': 5.0, 'Natural Gas Conversion': 1.0}, 'Fossil Gas': {'Hydrogen (Blue Hydrogen)': 2.0, 'Hydrogen (Green Hydrogen)': 4.0}, 'Fossil Oil': {'Biomass': 1.6}, 'Gas Plants': {'Carbon Capture and Storage (CCS)': 2.5}, 'Coal Plant': {'Ammonia Production': 10.0}}


In [ ]:
# just keep the relevant one for now hardcoded
#repurposing_costs_per_mw = {'Fossil Oil': {'Biomass': 1.6}}
#print(repurposing_costs_per_mw)

# Choose one demand scenario (here, clockwork_demand)
df = clockwork_demand

# Total demand per time step (MW)
scenario_total_demand = df.set_index("time")["total_demand_mw"].to_dict()

# Remove unnecessary columns and prepare technology-specific demand
df = df.drop(columns=["Unnamed: 0", "total_demand_mw"])
scenario_demand_per_type = {
    column: {str(row["time"]): row[column] for _, row in df.iterrows()}
    for column in df.columns if column != "time"
}

# Create list of time steps (as strings)
time_steps = df["time"].astype(str).tolist()


# final_t = time_steps[-1]  # final time period (used for penalty calculation)

In [ ]:
# Define the optimisation model
model = ConcreteModel()

# Define decision variables

# Decommissioning actions
model.decom_coal = Var(coal_plants.keys(), time_steps, domain=Binary)
model.decom_oil = Var(oil_plants.keys(), time_steps, domain=Binary)
model.decom_gas = Var(gas_plants.keys(), time_steps, domain=Binary)
model.decom_biomass = Var(biomass_plants.keys(), time_steps, domain=Binary)

# enhancement of renewable capacity
model.enhance_hydro_river = Var(hydro_river_plants.keys(), time_steps, domain=Binary)
model.enhance_nuclear = Var(nuclear_plants.keys(), time_steps, domain=Binary)
model.enhance_wind_off = Var(wind_off_plants.keys(), time_steps, domain=Binary)
model.enhance_wind_on = Var(wind_on_plants.keys(), time_steps, domain=Binary)
#model.enhance_hydro_storage = Var(hydro_storage_plants.keys(), time_steps, domain=Binary)
#model.enhance_solar = Var(solar_plants.keys(), time_steps, domain=Binary) # if global data are included

# Repurposing actions
model.coal_to_biomass = Var(coal_plants.keys(), time_steps, domain=Binary)
model.coal_to_nuclear = Var(coal_plants.keys(), time_steps, domain=Binary)
model.oil_to_biomass = Var(oil_plants.keys(), time_steps, domain=Binary)

# TODO: adding new plants (if so)

# --- Slack Variables for “Soft” Demand Requirements ---
# For technologies where effective capacity must not exceed a target (e.g. fossil fuels)
model.excess_coal = Var(time_steps, domain=NonNegativeReals)
model.excess_oil = Var(time_steps, domain=NonNegativeReals)
model.excess_gas = Var(time_steps, domain=NonNegativeReals)
model.excess_biomass = Var(time_steps, domain=NonNegativeReals)

# For technologies where effective capacity must meet a minimum (e.g. renewables) we allow shortfall
model.shortfall_nuclear = Var(time_steps, domain=NonNegativeReals)
model.shortfall_wind_off = Var(time_steps, domain=NonNegativeReals)
model.shortfall_wind_on = Var(time_steps, domain=NonNegativeReals)
model.shortfall_hydro_river = Var(time_steps, domain=NonNegativeReals)
model.shortfall_total = Var(time_steps, domain=NonNegativeReals)

In [ ]:
# --- Objective Function ---
# Compute the base cost of actions
def total_cost(model):
    cost_of_decommissioning = (
        # coal
        sum(
        coal_plants[p] * decommissioning_costs_per_mw["Fossil Hard coal"] * model.decom_coal[p, t]
        for p in coal_plants for t in time_steps
        ) +
        # oil
        sum(
        oil_plants[p] * decommissioning_costs_per_mw["Fossil Oil"] * model.decom_oil[p, t]
        for p in oil_plants for t in time_steps
        ) +
        # gas
        sum(
        gas_plants[p] * decommissioning_costs_per_mw["Fossil Gas"] * model.decom_gas[p, t]
        for p in gas_plants for t in time_steps
        ) +
        # biomass
        sum(
        biomass_plants[p] * decommissioning_costs_per_mw["Biomass"] * model.decom_biomass[p, t]
        for p in biomass_plants for t in time_steps
        )
    )
    cost_of_enhancing = (
        sum(
        hydro_river_plants[w] * enhancements_cost_per_mw['Hydro Run-of-river and poundage'] *
            model.enhance_hydro_river[w, t]
        for w in hydro_river_plants for t in time_steps
        ) +
        sum(
        nuclear_plants[w] * enhancements_cost_per_mw["Nuclear"] * model.enhance_nuclear[w, t]
        for w in nuclear_plants for t in time_steps
        ) +
        sum(
        wind_off_plants[w] * enhancements_cost_per_mw["Wind Offshore"] * model.enhance_wind_off[w, t]
        for w in wind_off_plants for t in time_steps
        ) +
        sum(
        wind_on_plants[w] * enhancements_cost_per_mw["Wind Onshore"] * model.enhance_wind_on[w, t]
        for w in wind_on_plants for t in time_steps
        )
        # +
        # sum(
        # hydro_storage_plants[w] * enhancements_cost_per_mw['Hydro Pumped Storage'] *
        #     model.enhance_hydro_storage[w, t]
        # for w in hydro_storage_plants for t in time_steps
        # )
    )
    cost_of_repurposing = (
        sum(
        coal_plants[p] * repurposing_costs_per_mw["Fossil Hard coal"]["Biomass"] *
        model.coal_to_biomass[p, t]
        for p in coal_plants for t in time_steps
        ) +
        sum(
        coal_plants[p] * repurposing_costs_per_mw["Fossil Hard coal"]["Nuclear"] *
        model.coal_to_nuclear[p, t]
        for p in coal_plants for t in time_steps
        ) +
        sum(
        oil_plants[p] * repurposing_costs_per_mw["Fossil Oil"]["Biomass"] *
        model.oil_to_biomass[p, t]
        for p in oil_plants for t in time_steps
        )
    )
    total_cost = cost_of_decommissioning + cost_of_enhancing + cost_of_repurposing
    return total_cost

In [ ]:
# Define the penalty cost per MW of unmet/excess capacity (in millions GBP per MW)
PENALTY_COST_PER_MW_non_renew  = 1000 #here
PENALTY_COST_PER_MW_renew = 10 #here
PENALTY_COST_PER_MW_total_demand = 10

# Instead of summing the slack at every time step, we penalize only the final (cumulative) gap.
def total_cost_with_penalty(model):
    base_cost = total_cost(model)
    # Sum penalties for non-renewables over all time periods
    penalty_non_renewables = PENALTY_COST_PER_MW_non_renew * sum(
         model.excess_coal[t] +
         model.excess_oil[t] +
         model.excess_gas[t] +
         model.excess_biomass[t] for t in time_steps
    )
    # Sum penalties for renewables/shortfall over all time periods
    penalty_renewables = PENALTY_COST_PER_MW_renew * sum(
         model.shortfall_nuclear[t] +
         model.shortfall_wind_off[t] +
         model.shortfall_wind_on[t] +
         model.shortfall_hydro_river[t] for t in time_steps
    )
    penalty_total_demand = PENALTY_COST_PER_MW_total_demand * sum(
        model.shortfall_total[t] for t in time_steps
    )
    return base_cost + penalty_non_renewables + penalty_renewables + penalty_total_demand

model.objective = Objective(rule=total_cost_with_penalty, sense=minimize)

In [ ]:
# Constraints

# One action per plant constraint over the planning horizon

# coal
def single_action_per_coal_plant_constraint(model, p):
    return sum(model.decom_coal[p, t] + model.coal_to_biomass[p, t] +
               model.coal_to_nuclear[p, t]
               for t in time_steps) <= 1
model.single_action_per_coal_plant_constraint = Constraint(coal_plants.keys(), rule=single_action_per_coal_plant_constraint)

# biomass
def single_action_per_biomass_plant_constraint(model, p):
    return sum(model.decom_biomass[p, t] for t in time_steps) <= 1
model.single_action_per_biomass_plant_constraint = Constraint(biomass_plants.keys(), rule=single_action_per_biomass_plant_constraint)

# oil
def single_action_per_oil_plant_constraint(model, p):
    return sum(model.decom_oil[p, t] + model.oil_to_biomass[p, t] for t in time_steps) <= 1
model.single_action_per_oil_plant_constraint = Constraint(oil_plants.keys(), rule=single_action_per_oil_plant_constraint)

# gas
def single_action_per_gas_plant_constraint(model, p):
    return sum(model.decom_gas[p, t] for t in time_steps) <= 1
model.single_action_per_gas_plant_constraint = Constraint(gas_plants.keys(), rule=single_action_per_gas_plant_constraint)

# wind_off plant
def single_action_per_wind_off_plant_constraint(model, p):
    return sum(model.enhance_wind_off[p, t] for t in time_steps) <= 1
model.single_action_per_wind_off_plant_constraint = Constraint(wind_off_plants.keys(), rule=single_action_per_wind_off_plant_constraint)

# wind_on plant
def single_action_per_wind_on_plant_constraint(model, p):
    return sum(model.enhance_wind_on[p, t] for t in time_steps) <= 1
model.single_action_per_wind_on_plant_constraint = Constraint(wind_on_plants.keys(), rule=single_action_per_wind_on_plant_constraint)

# hydro_river plant
def single_action_per_hydro_river_plant_constraint(model, p):
    return sum(model.enhance_hydro_river[p, t] for t in time_steps) <= 1
model.single_action_per_hydro_river_plant_constraint = Constraint(hydro_river_plants.keys(),
                                                                  rule=single_action_per_hydro_river_plant_constraint)

# # hydro_storage plant
# def single_action_per_hydro_storage_plant_constraint(model, p):
#     return sum(model.enhance_hydro_storage[p, t] for t in time_steps) <= 1
# model.single_action_per_hydro_storage_plant_constraint = Constraint(hydro_storage_plants.keys(),
#                                                                     rule=single_action_per_hydro_storage_plant_constraint)

# nuclear plant
def single_action_per_nuclear_plant_constraint(model, p):
    return sum(model.enhance_nuclear[p, t] for t in time_steps) <= 1
model.single_action_per_nuclear_plant_constraint = Constraint(nuclear_plants.keys(),
                                                              rule=single_action_per_nuclear_plant_constraint)

In [ ]:
# --- "Soft" Demand Constraints via Slack Variables ---

# For fossil fuels (upper-bound constraints) we allow some excess capacity but penalize it.
def coal_slack_constraint(model, t):
    effective_coal = sum(
        coal_plants[p] * (1 - sum(model.decom_coal[p, tp] + model.coal_to_biomass[p, tp] + model.coal_to_nuclear[p, tp]
                                   for tp in time_steps if tp <= t))
        for p in coal_plants
    )
    return effective_coal - model.excess_coal[t] <= scenario_demand_per_type["Fossil Hard coal"][t]
model.coal_slack_constraint = Constraint(time_steps, rule=coal_slack_constraint)

def oil_slack_constraint(model, t):
    effective_oil = sum(
        oil_plants[p] * (1 - sum(model.decom_oil[p, tp] + model.oil_to_biomass[p, tp]
                                  for tp in time_steps if tp <= t))
        for p in oil_plants
    )
    return effective_oil - model.excess_oil[t] <= scenario_demand_per_type["Fossil Oil"][t]
model.oil_slack_constraint = Constraint(time_steps, rule=oil_slack_constraint)

def gas_slack_constraint(model, t):
    effective_gas = sum(
        gas_plants[p] * (1 - sum(model.decom_gas[p, tp]
                                  for tp in time_steps if tp <= t))
        for p in gas_plants
    )
    return effective_gas - model.excess_gas[t] <= scenario_demand_per_type["Fossil Gas"][t]
model.gas_slack_constraint = Constraint(time_steps, rule=gas_slack_constraint)

def biomass_slack_constraint(model, t):
    effective_biomass = (
        sum(
            biomass_plants[p] * (1 - sum(model.decom_biomass[p, tp]
                                         for tp in time_steps if tp <= t))
            for p in biomass_plants
        ) +
        sum(
            coal_plants[p] * sum(model.coal_to_biomass[p, tp] for tp in time_steps if tp <= t)
            for p in coal_plants
        ) +
        sum(
            oil_plants[p] * sum(model.oil_to_biomass[p, tp] for tp in time_steps if tp <= t)
            for p in oil_plants
        )
    )
    return effective_biomass - model.excess_biomass[t] <= scenario_demand_per_type["Biomass"][t]
model.biomass_slack_constraint = Constraint(time_steps, rule=biomass_slack_constraint)

# For technologies with minimum capacity requirements, we allow shortfall but penalize it.
def nuclear_shortfall_constraint(model, t):
    effective_nuclear = sum(
        nuclear_plants[w] + sum(model.enhance_nuclear[w, tw] * enhancement_percentage["Nuclear"] * nuclear_plants[w]
                                 for tw in time_steps if tw <= t)
        for w in nuclear_plants
    ) + sum(
        coal_plants[p] * sum(model.coal_to_nuclear[p, tp] for tp in time_steps if tp <= t)
        for p in coal_plants
    )
    return effective_nuclear + model.shortfall_nuclear[t] >= scenario_demand_per_type["Nuclear"][t]
model.nuclear_shortfall_constraint = Constraint(time_steps, rule=nuclear_shortfall_constraint)

def wind_off_shortfall_constraint(model, t):
    effective_wind_off = sum(
        wind_off_plants[w] + sum(model.enhance_wind_off[w, tw] * enhancement_percentage["Wind Offshore"] * wind_off_plants[w]
                                 for tw in time_steps if tw <= t)
        for w in wind_off_plants
    )
    return effective_wind_off + model.shortfall_wind_off[t] >= scenario_demand_per_type["Wind Offshore"][t]
model.wind_off_shortfall_constraint = Constraint(time_steps, rule=wind_off_shortfall_constraint)

def wind_on_shortfall_constraint(model, t):
    effective_wind_on = sum(
        wind_on_plants[w] + sum(model.enhance_wind_on[w, tw] * enhancement_percentage["Wind Onshore"] * wind_on_plants[w]
                                 for tw in time_steps if tw <= t)
        for w in wind_on_plants
    )
    return effective_wind_on + model.shortfall_wind_on[t] >= scenario_demand_per_type["Wind Onshore"][t]
model.wind_on_shortfall_constraint = Constraint(time_steps, rule=wind_on_shortfall_constraint)

def hydro_river_shortfall_constraint(model, t):
    effective_hydro_river = sum(
        hydro_river_plants[w] + sum(model.enhance_hydro_river[w, tw] *
                                     enhancement_percentage['Hydro Run-of-river and poundage'] *
                                     hydro_river_plants[w]
                                     for tw in time_steps if tw <= t)
        for w in hydro_river_plants
    )
    return effective_hydro_river + model.shortfall_hydro_river[t] >= scenario_demand_per_type['Hydro Run-of-river and poundage'][t]
model.hydro_river_shortfall_constraint = Constraint(time_steps, rule=hydro_river_shortfall_constraint)

In [ ]:
def total_demand_shortfall_constraint(model, t):
    effective_coal = sum(
        coal_plants[p] * (1 - sum(model.decom_coal[p, tp] + model.coal_to_biomass[p, tp] + model.coal_to_nuclear[p, tp]
                                   for tp in time_steps if tp <= t))
        for p in coal_plants
    )
    effective_oil = sum(
        oil_plants[p] * (1 - sum(model.decom_oil[p, tp] + model.oil_to_biomass[p, tp]
                                  for tp in time_steps if tp <= t))
        for p in oil_plants
    )
    effective_gas = sum(
        gas_plants[p] * (1 - sum(model.decom_gas[p, tp]
                                  for tp in time_steps if tp <= t))
        for p in gas_plants
    )
    effective_biomass = (
        sum(
            biomass_plants[p] * (1 - sum(model.decom_biomass[p, tp]
                                         for tp in time_steps if tp <= t))
            for p in biomass_plants
        ) +
        sum(
            coal_plants[p] * sum(model.coal_to_biomass[p, tp] for tp in time_steps if tp <= t)
            for p in coal_plants
        ) +
        sum(
            oil_plants[p] * sum(model.oil_to_biomass[p, tp] for tp in time_steps if tp <= t)
            for p in oil_plants
        )
    )
    effective_nuclear = sum(
        nuclear_plants[w] + sum(model.enhance_nuclear[w, tw] * enhancement_percentage["Nuclear"] * nuclear_plants[w]
                                 for tw in time_steps if tw <= t)
        for w in nuclear_plants
    ) + sum(
        coal_plants[p] * sum(model.coal_to_nuclear[p, tp] for tp in time_steps if tp <= t)
        for p in coal_plants
    )
    effective_wind_off = sum(
        wind_off_plants[w] + sum(model.enhance_wind_off[w, tw] * enhancement_percentage["Wind Offshore"] * wind_off_plants[w]
                                 for tw in time_steps if tw <= t)
        for w in wind_off_plants
    )
    effective_wind_on = sum(
        wind_on_plants[w] + sum(model.enhance_wind_on[w, tw] * enhancement_percentage["Wind Onshore"] * wind_on_plants[w]
                                 for tw in time_steps if tw <= t)
        for w in wind_on_plants
    )
    effective_hydro_river = sum(
        hydro_river_plants[w] + sum(model.enhance_hydro_river[w, tw] *
                                     enhancement_percentage['Hydro Run-of-river and poundage'] *
                                     hydro_river_plants[w]
                                     for tw in time_steps if tw <= t)
        for w in hydro_river_plants
    )

    total_effective = (effective_coal + effective_oil + effective_gas +
                       effective_biomass + effective_nuclear +
                       effective_wind_off + effective_wind_on +
                       effective_hydro_river)

    return total_effective + model.shortfall_total[t] >= scenario_total_demand[t]
model.total_demand_shortfall_constraint = Constraint(time_steps, rule=total_demand_shortfall_constraint)

In [ ]:
solver = SolverFactory("glpk")
solver.options["tmlim"] = 30  # Time limit in seconds

results = solver.solve(model, tee=True)

# TODO: results visualization

In [ ]:
# # ---------------------------------
# # 4. REPORT RESULTS
# # ---------------------------------

print("Objective Value (Total Cost):", value(model.objective))
for t in time_steps:
    penalty_non_renew = PENALTY_COST_PER_MW_non_renew * (
         value(model.excess_coal[t]) +
         value(model.excess_oil[t]) +
         value(model.excess_gas[t]) +
         value(model.excess_biomass[t])
    )
    penalty_renew = PENALTY_COST_PER_MW_renew * (
         value(model.shortfall_nuclear[t]) +
         value(model.shortfall_wind_off[t]) +
         value(model.shortfall_wind_on[t]) +
         value(model.shortfall_hydro_river[t]) +
         value(model.shortfall_total[t])
    )
    print(f"Time {t}:")
    print("  Penalty for Excess =", penalty_non_renew)
    print("  Penalty for Shortfall =", penalty_renew)

In [ ]:
# ---------------------------------
# 4. REPORT RESULTS
# ---------------------------------
final_t = "2020"
print("Objective Value (Total Cost):", value(model.objective))
print("Penalties are applied only on the final period (", final_t, ").")
print("Final period slack values:")
print("  Excess Coal =", value(model.excess_coal[final_t]))
print("  Excess Oil  =", value(model.excess_oil[final_t]))
print("  Excess Gas  =", value(model.excess_gas[final_t]))
print("  Excess Biomass =", value(model.excess_biomass[final_t]))
print("  Shortfall Nuclear =", value(model.shortfall_nuclear[final_t]))
print("  Shortfall Wind Offshore =", value(model.shortfall_wind_off[final_t]))
print("  Shortfall Wind Onshore  =", value(model.shortfall_wind_on[final_t]))
print("  Shortfall Hydro Run-of-river =", value(model.shortfall_hydro_river[final_t]))
print("  Total Shortfall =", value(model.shortfall_total[final_t]))

In [ ]:
# --- After solving the model, extract slack values per time step for all energy categories ---
import pandas as pd

# Build a dictionary where each key corresponds to a slack variable (or "excess" for fossil fuels)
slack_data = {
    "Time": time_steps,
    "Excess Coal": [value(model.excess_coal[t]) for t in time_steps],
    "Excess Oil": [value(model.excess_oil[t]) for t in time_steps],
    "Excess Gas": [value(model.excess_gas[t]) for t in time_steps],
    "Excess Biomass": [value(model.excess_biomass[t]) for t in time_steps],
    "Shortfall Nuclear": [value(model.shortfall_nuclear[t]) for t in time_steps],
    "Shortfall Wind Offshore": [value(model.shortfall_wind_off[t]) for t in time_steps],
    "Shortfall Wind Onshore": [value(model.shortfall_wind_on[t]) for t in time_steps],
    "Shortfall Hydro Run-of-river": [value(model.shortfall_hydro_river[t]) for t in time_steps],
    "Total Shortfall": [value(model.shortfall_total[t]) for t in time_steps]
}

# Convert the dictionary into a pandas DataFrame
df_slack = pd.DataFrame(slack_data)

# Display the DataFrame
print(df_slack)


In [ ]:
# Calculate total effective capacity from each technology (this depends on your model's definition)
# For illustration, suppose you define:
total_effective = { t: (effective_coal(t) + effective_oil(t) + effective_gas(t) +
                        effective_biomass(t) + effective_nuclear(t) +
                        effective_wind_off(t) + effective_wind_on(t) +
                        effective_hydro_river(t))
                    for t in time_steps }

# Then, the net shortfall (if any) at time t could be computed as:
net_shortfall = { t: max(0, scenario_total_demand[t] - total_effective[t]) for t in time_steps }

print(net_shortfall)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import value

# Prepare data
time_labels = list(time_steps)
x = np.arange(len(time_labels))  # Time indices

# Extract number of actions per time step
decom_vals = [sum(value(model.decom_coal[p, t]) for p in coal_plants) +
              sum(value(model.decom_oil[p, t]) for p in oil_plants) +
              sum(value(model.decom_gas[p, t]) for p in gas_plants) +
              sum(value(model.decom_biomass[p, t]) for p in biomass_plants)
              for t in time_steps]

enhance_vals = [sum(value(model.enhance_hydro_river[w, t]) for w in hydro_river_plants) +
                sum(value(model.enhance_nuclear[w, t]) for w in nuclear_plants) +
                sum(value(model.enhance_wind_off[w, t]) for w in wind_off_plants) +
                sum(value(model.enhance_wind_on[w, t]) for w in wind_on_plants)
                for t in time_steps]

repurpose_vals = [sum(value(model.coal_to_biomass[p, t]) for p in coal_plants) +
                  sum(value(model.coal_to_nuclear[p, t]) for p in coal_plants) +
                  sum(value(model.oil_to_biomass[p, t]) for p in oil_plants)
                  for t in time_steps]

# Plot grouped bar chart
fig, ax = plt.subplots(figsize=(10, 5))
width = 0.2  # Width of each bar

ax.bar(x - width, decom_vals, width=width, label="Decommissioned", color="red")
ax.bar(x, enhance_vals, width=width, label="Enhanced", color="green")
ax.bar(x + width, repurpose_vals, width=width, label="Repurposed", color="blue")

# Add labels and title
ax.set_xlabel("Time Steps")
ax.set_ylabel("Number of Actions")
ax.set_title("Power Plant Actions Over Time")
ax.set_xticks(x)
ax.set_xticklabels(time_labels)  # Set time step labels
ax.legend()

# Show the plot
plt.show()
